In [5]:
# import dependencies

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas import DataFrame

In [7]:
html = "https://www.zip-codes.com/county/ca-san-diego.asp"
contents = requests.get(html)

soup = BeautifulSoup(contents.content, 'lxml')
# Need to parse through DOM to know which table to scrape
table = soup.find_all('table')[2]
lists = pd.read_html(str(table))

In [12]:
website_data_df = lists[0]
print(f'Initial size of dataframe: {website_data_df.shape}')
# Set column headers
website_data_df.columns = website_data_df.iloc[0]
website_data_df = website_data_df.drop(0)
# Reset index 
website_data_df = website_data_df.reset_index(drop=True)
website_data_df.head()

Initial size of dataframe: (182, 6)


,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
0,ZIP Code 91901,General,Alpine,17403,Pacific,619
1,ZIP Code 91902,General,Bonita,17653,Pacific,619
2,ZIP Code 91903,P.O. Box,Alpine,0,Pacific,619
3,ZIP Code 91905,General,Boulevard,1700,Pacific,619/760
4,ZIP Code 91906,General,Campo,3627,Pacific,619


In [18]:
# Drop Zip Codes with Population 0
no_population = website_data_df[website_data_df["Population"] == '0'].index
website_data_df.drop(no_population, inplace=True)
print(f'Updated size of dataframe: {website_data_df.shape[0]} rows and {website_data_df.shape[1]} columns')
# Drop "Zip Code" phrase in Zip Code column
website_data_df["ZIP Code"] = website_data_df["ZIP Code"].str[-5:]
website_data_df.head()

Updated size of dataframe: 106 rows and 6 columns


,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
0,91901,General,Alpine,17403,Pacific,619
1,91902,General,Bonita,17653,Pacific,619
3,91905,General,Boulevard,1700,Pacific,619/760
4,91906,General,Campo,3627,Pacific,619
7,91910,General,Chula Vista,75802,Pacific,619
